       1_6_230104_015_cafe_display
         
         4-7 section
         
         data : /aiffel/socar_gis/data/seoul_store_info_202106.csv

In [1]:
#1. package load
import requests
import json
import os
import math
import time
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt

from shapely.geometry import mapping, shape, Point, Polygon, LineString
from folium import plugins
from folium.plugins import MarkerCluster, HeatMap

print(pd.__version__)
print(gpd.__version__)
print(folium.__version__)

1.4.4
0.12.2
0.14.0


    2. 프로젝트(1) 서울시 쏘카존 데이터를 활용한 시각화
       1) data load : socar zone data
       2) 시각화
          - base nap : 서울시 중심부 위도, 경도
          - MarkerCluster 객체 
          - map draw

In [2]:
#2. 프로젝트(1)  ... data load
path = os.getenv('HOME') + '/aiffel/socar_gis/data/20210823_socar_zone.csv'

socar_zone_info_df = pd.read_csv(path)
socar_zone_info_df.head()

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [ ]:
#2. 프로젝트(1)  # base nap : 서울시 중심부 위도, 경도
center = [37.541, 126.986]

map = folium.Map(
    location=center,
    zoom_start=10,
    )
print('슝~')

In [ ]:
#2. 프로젝트(1)  # MarkerCluster 객체 
marker_cluster = MarkerCluster().add_to(map)

#maker 객체 추가
for idx, row in socar_zone_info_df.iterrows():
    try:
        folium.Marker(
            location=[socar_zone_info_df.loc[idx, "lat"], socar_zone_info_df.loc[idx, "lng"]],
            popup=socar_zone_info_df.loc[idx, "zone_name"],
            icon=folium.Icon(color="cadetblue", icon="car", prefix="fa")            
        ).add_to(marker_cluster)
    except:
        pass

print('슝~')

In [ ]:
#2. 프로젝트(1)  # map draw
map

    3. 프로젝트(2) 서울시 카페와 버스 정류장 밀도 데이터를 이용한 시각화
           1) 카페data 정리
              - data load
              - 카페data 전처리 : 업종data분류
              - 카페 위치데이터 리스트 만들기
           2) 버스data 정리
              - data load
              - 버스data 전처리 .. 컬럼명 변경
              - 버스 정류장 위치데이터 리스트 만들기
           3) 시각화 
              - Folium 초기 Map 객체 생성
              - 레이어 적용을 위한 그룹 만들기
              - 카페 밀도
              - 버스정류장 밀도
           4) 생성한 map 확인

In [ ]:
#3. 프로젝트(2) ... 1) 카페data load
temp_seoul_store_df = pd.read_csv(os. getenv('HOME') + '/aiffel/socar_gis/data/seoul_store_info_202106.csv')

temp_seoul_store_df.head(3)

In [ ]:
#3. 프로젝트(2) ... 1) 카페data 전처리 
seoul_cafe_store_df = temp_seoul_store_df[temp_seoul_store_df["상권업종소분류코드"] == "Q12A01"]

seoul_cafe_store_df.head()

In [ ]:
#3. 프로젝트(2) ... 1) 카페data 전처리 확인 
seoul_cafe_store_df.info()

In [ ]:
#3. 프로젝트(2) ... 1) 카페data 전처리 ... # 카페 위치데이터 리스트 만들기
seoul_cafe_location_data = [[row["위도"], row["경도"]] for _, row in seoul_cafe_store_df.iterrows()]

print("슝~")

In [ ]:
#3. 프로젝트(2) ... 3) 시각화 # 레이어 적용을 위한 그룹 만들기
# 대표 그룹 만들기
fg = folium.FeatureGroup(name="groups")
map.add_child(fg)

# 카페 그룹 만들기
cafe = plugins.FeatureGroupSubGroup(fg, "cafe_density")
map.add_child(cafe)

# # 버스정류장 그룹 만들기
# bus = plugins.FeatureGroupSubGroup(fg, "bus_group")
# map.add_child(bus)

# 레이어 콘트롤 박스 만들기
folium.LayerControl(collapsed=False).add_to(map)

In [ ]:
#3. 프로젝트(2) ... 3) 시각화 .. # 카페 밀도
plugins.HeatMap(seoul_cafe_location_data).add_to(cafe)

In [ ]:
#3. 프로젝트(2) ... 3) 시각화 ... # 생성한 맵 확인
map